In [3]:
from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load the dataset
eeg_data = pd.read_csv('EEGdata_in_time.csv', header=None)

# Rename the header of the 9th column (index 9)
eeg_data.rename(columns={eeg_data.columns[9]: 'Time'}, inplace=True)

# Define the sampling rate (250 Hz)
sampling_rate = 250  # Hz
time_increment = 1 / sampling_rate  # seconds per sample

# Define the bandpass filter parameters
low_cutoff = 0.5  # Low cutoff frequency in Hz
high_cutoff = 30  # High cutoff frequency in Hz

# Design the Butterworth bandpass filter
b, a = butter(
    N=4,  # 4th order filter
    Wn=[low_cutoff / (sampling_rate / 2), high_cutoff / (sampling_rate / 2)],  # Normalize frequencies
    btype='bandpass',  # Bandpass filter
    analog=False
)

# Apply the bandpass filter to the electrode columns (0 to 7)
filtered_data = eeg_data.copy()
for col in range(8):  # Assuming electrodes are in columns 0 to 7
    filtered_data.iloc[:, col] = filtfilt(b, a, eeg_data.iloc[:, col])

# Save the filtered data to a new CSV file
filtered_csv_path = 'EEGdata_filtered_bandpass.csv'
filtered_data.to_csv(filtered_csv_path, index=False, header=False)

Now let's generate the plots after the Butterworth filtering:

In [4]:
# Create a directory to save the plots
output_dir = "trigger_plots_filtered"
os.makedirs(output_dir, exist_ok=True)

# Track previously processed ranges to avoid duplicate or overlapping plots
processed_ranges = []

# Assuming column 8 contains the trigger values
trigger_column_index = 8  # Adjust if the trigger column index is different

# Identify the indices where triggers occur (non-zero values in the trigger column)
trigger_occurrences = eeg_data[eeg_data.iloc[:, trigger_column_index] != 0].index

# Calculate 0.1 seconds (25 samples) before and 0.4 seconds (100 samples) after the first trigger
samples_before_trigger = int(0.1 * sampling_rate)
samples_after_trigger = int(0.4 * sampling_rate)

# Loop through all trigger occurrences
for i, trigger_idx in enumerate(trigger_occurrences):
    # Calculate the zoomed-in range for the current trigger
    start_idx = max(0, trigger_idx - samples_before_trigger)
    end_idx = min(len(eeg_data), trigger_idx + samples_after_trigger)

    # Skip if this range overlaps with a previously processed range
    if any(start_idx <= end <= end_idx or start_idx <= start <= end_idx for (start, end) in processed_ranges):
        continue

    # Extract the data for the zoomed-in region
    zoomed_data = eeg_data.iloc[start_idx:end_idx + 1]

    # Get the trigger value and time
    trigger_value = eeg_data.iloc[trigger_idx, trigger_column_index]
    trigger_time = eeg_data['Time'].iloc[trigger_idx]

    # Create the plot
    plt.figure(figsize=(20, 6))
    plt.plot(zoomed_data['Time'], zoomed_data.iloc[:, 0], label='Electrode 1', color='blue')
    plt.plot(zoomed_data['Time'], zoomed_data.iloc[:, 1], label='Electrode 2', color='green')
    plt.plot(zoomed_data['Time'], zoomed_data.iloc[:, 2], label='Electrode 3', color='yellow')
    plt.plot(zoomed_data['Time'], zoomed_data.iloc[:, 3], label='Electrode 4', color='purple')
    plt.plot(zoomed_data['Time'], zoomed_data.iloc[:, 4], label='Electrode 5', color='orange')
    plt.plot(zoomed_data['Time'], zoomed_data.iloc[:, 5], label='Electrode 6', color='cyan')
    plt.plot(zoomed_data['Time'], zoomed_data.iloc[:, 6], label='Electrode 7', color='magenta')
    plt.plot(zoomed_data['Time'], zoomed_data.iloc[:, 7], label='Electrode 8', color='brown')
    plt.plot(zoomed_data['Time'], zoomed_data.iloc[:, 8], label='Triggers', color='red', linestyle='--', alpha=0.7)

    # Add a vertical line at the trigger occurrence
    plt.axvline(
        x=trigger_time,
        color='red',
        linestyle='-',
        linewidth=1.5,
        label=f'Trigger {int(trigger_value)}'
    )

    # Remove close ticks to avoid overlap
    current_ticks = plt.xticks()[0]
    new_ticks = [tick for tick in current_ticks if abs(tick - trigger_time) > 0.02]  # Remove ticks within 0.02s of trigger time
    new_ticks.append(trigger_time)
    plt.xticks(new_ticks, labels=[f'{tick:.3f}' for tick in new_ticks], rotation=45)

    # Add labels, title, and legend
    plt.xlabel('Time (s)')
    plt.ylabel('Signal Amplitude / Trigger Value')
    plt.title(f'Trigger {i + 1}: Value = {int(trigger_value)}')
    plt.legend()
    plt.grid()

    # Save the plot
    plt.savefig(os.path.join(output_dir, f'trigger_{i + 1}_value_{int(trigger_value)}.png'))
    plt.close()

    # Add this range to processed_ranges
    processed_ranges.append((start_idx, end_idx))

# Indicate completion
print(f"Filtered plots for all unique triggers with optimized x-axis labels have been saved in the '{output_dir}' directory.")

Filtered plots for all unique triggers with optimized x-axis labels have been saved in the 'trigger_plots_filtered' directory.
